In [ ]:
import sys

!{sys.executable} -m pip install keyring artifacts-keyring
!{sys.executable} -m pip install --pre --upgrade --trusted-host pkgs.dev.azure.com --trusted-host pypi.org --trusted-host "*.blob.core.windows.net" --trusted-host files.pythonhosted.org --extra-index-url https://pkgs.dev.azure.com/mathco-products/_packaging/pip-codex-wf%40Local/pypi/simple/ "codex-widget-factory<=0.1"



In [ ]:
# tags to identify this iteration when submitted
# example: codex_tags = {'env': 'dev', 'region': 'USA', 'product_category': 'A'}

codex_tags = {
}

from codex_widget_factory import utils
results_json=[]

import pandas as pd
import numpy as np

## Ingestion File System

In [ ]:
#Begin code here
#File ingestion from azure blob storage using connection credentials

from codex_widget_factory.ingestion.file_system import get_ingested_data as ingestion_file_system_get_ingested_data

response_0 = ingestion_file_system_get_ingested_data(file_path='Sales_Data.csv',
                                                     datasource_type='azure_blob_storage',
                                                     connection_uri='DefaultEndpointsProtocol=https;AccountName=coach;AccountKey=qtMAw9Z1T+1oOl+joMwfzdvnR0exMA4Qw50vniiaXOFvpOeFG7TP+g+DP4/iU7VKAUhirSpzrESvjF3U2Ld0KA==;EndpointSuffix=core.windows.net',
                                                     container_name='ddrs11/aegon_nischal')
results_json.append({
                        'type': 'Ingestion',
                        'name': 'File System',
                        'component': 'get_ingested_data',
                        'visual_results': utils.get_response_visuals(response_0),
                        'metrics': False
                    })


display (response_0)

#END WIDGET CODE



 ### Custom Data cleaning

In [ ]:
#BEGIN CUSTOM CODE BELOW...


# asking python to read the given values as missing values/ null values.
missing_value = ["nan","#DIV/0!",'Missing Values','Missing',' ','-','NA','N/A','NaN','na','#ref', np.nan] 

#response_0 = pd.read_csv('Sales_Data.csv', na_values = missing_value)
response_0 = response_0.replace(["nan","#DIV/0!",'Missing Values','Missing',' ','-','NA','N/A','NaN','na','#ref'],[np.nan]*11)

#Finding which columns has missing values
print('Columns and corresponding null values are:\n', response_0.isnull().sum().sort_values(ascending = False))

# Filling the NULL values in 'broker_percentage_lost_2yrs' with mean 
response_0['broker_percentage_lost_2yrs'].fillna(8.077246e+01,inplace = True)

# Finding mode values to fill na values in,'Gen_Channel', 'MTP_Channel','primary' columns
gen_channel = response_0['Gen_Channel'].mode()
mtp_channel= response_0['MTP_Channel'].mode()
primary= response_0['Primary_Excess'].mode()

#Filling null values for 'Gen_Channel', 'MTP_Channel' & 'Primary_Excess'
response_0['Gen_Channel'].fillna(gen_channel.values[0],inplace = True)
response_0['MTP_Channel'].fillna(mtp_channel.values[0],inplace = True)
response_0['Primary_Excess'].fillna(primary.values[0],inplace = True)

#Checking if any null values exist
print('After DATA CLEANING:\n The Sum of Null Values In The Data Set is:', response_0.isnull().sum().sum())

#dropping duplicate values
response_0=pd.concat([response_0["Opportunity_18_ID"].to_frame(),response_0.iloc[:,1:].drop_duplicates()],axis=1).dropna()

#changing the format of date column from string to datetime
response_0['Inception_Renewal_Date'] = pd.to_datetime(response_0['Inception_Renewal_Date'])

#put your output in this response param for connecting to downstream widgets
response_1 = response_0

#END CUSTOM CODE



## Transformation Remove Outliers

In [ ]:
#BEGIN WIDGET CODE BELOW...
# This function removes the outliers that are present in the column/dataset
# Based on the interquartile range or replaces them with NaNs.
    

from codex_widget_factory.transformation.remove_outliers import get_transformed_data as transformation_remove_outliers_get_transformed_data
response_2 = transformation_remove_outliers_get_transformed_data(df = response_1, dep_var = 'Won',col_names = ['gwp_lost_2yrs','gwp_won_2yrs','num_won_2yrs','num_lost_2yrs','num_uniqprod_2yrs','num_won_2yrs',
                                                                                                               'num_lost_2yrs','num_wonnew_2yrs','num_quot_2yrs','num_existing','broker_contact_num_won_2yrs',
                                                                                                               'broker_contact_days_since_last_lost','quot_not_won','num_lost_wo_subreceieved_2yrs',
                                                                                                               'num_lost_postsub_2yrs','num_lost_postquot_2yrs','broker_days_since_last_won','broker_num_won_2yrs'])
results_json.append({
                      'type': 'Transformation',
                      'name': 'Remove Outliers',
                      'component': 'get_transformed_data',
                      'visual_results': utils.get_response_visuals(response_2),
                      'metrics': False
                    })
display(response_2)

#utils.render_response(response_2)

#END WIDGET CODE




## Custom Bivariate

In [ ]:
##BIVARIATE ANALYSIS
#custom code for bivariate analysis report
bivariate_factors = response_1.copy()
bivariate_factors["temp"] = 1
import plotly.express as px

def percentage_pivot(col,data):
    '''
    Bivariate Analysis
    
    Parameters:
    col:Factors impacting the output variable
    
    Returns:
    Bivariate charts for the given list of attributes
    
    '''
    
    data_pivot = data.groupby([col,"Stage"], as_index = False)['temp'].sum()
    data_pivot1= data.groupby([col], as_index = False)['temp'].sum()
    data_pivot1 = data_pivot1.rename(columns = {"temp":"sum"})
    data_pivot = pd.merge(data_pivot,data_pivot1,on = col,how = "inner")
    data_pivot["Won"] = data_pivot["temp"]/data_pivot["sum"]
    return data_pivot

#Tuple of factors that influence the output variable
factors_list=('Line_of_Business','Appetite', 'OppCountry','Business_Type','Entity_Hierarchy_Level','Type','Entity_Record_Type','num_won_2yrs','num_uniqprod_2yrs',
        'Gen_Channel','Product')


def bivariate(element):
    """
    Bivariate analysis
    
    Parameters:
    Element:Factors from the tuple
    
    Returns:
    Iterates through the tuple and returns bivariate charts for the given list of attributes
    
    """
    temp_df = percentage_pivot(col = element, data = bivariate_factors)
    print(temp_df)
    fig = px.bar(temp_df, x = element,y= "Won", barmode = 'stack',color = "Stage")
    results_json.append({
                          'type': 'Exploration Analysis',
                          'name': 'Bivariate Report',
                          'component': 'Stacked_'+str(element),
                          'visual_results': utils.get_response_visuals({"stacked":fig}),
                          'metrics': False
                        })
    fig.show()
    
list(map(bivariate,factors_list))